In [1]:
import os
import firebirdsql
import pandas as pd
import numpy as np
import xlsxwriter
import datetime
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

True

In [2]:
def get_firebird_connection():
    return firebirdsql.connect(
        host=os.getenv('HOST'),
        port=int(os.getenv('PORT', '3050')),
        database=os.getenv('DB_PATH'),
        user=os.getenv('APP_USER'),
        password=os.getenv('PASSWORD'),
        role=os.getenv('ROLE'),
        auth_plugin_name=os.getenv('AUTH'),
        wire_crypt=False,
        charset='ISO8859_1'
    )

In [8]:
filename = "/Users/mateusspinola/Desktop/A_FATURAR-14_04.xlsx"

# Lê a planilha a partir da linha 8 (ignorando as 7 primeiras linhas)
df = pd.read_excel(filename, skiprows=7)

# Converte a coluna DATA para datetime, considerando o formato dd/mm/yy (dayfirst=True)
df['DATA'] = pd.to_datetime(df['DATA'], dayfirst=True)

# Ordena por CLIENTE e DATA
df = df.sort_values(['CLIENTE', 'DATA'])

def agrupar_pedidos(grupo):
    """
    Agrupa os pedidos de um cliente em grupos onde a diferença entre
    a data do primeiro pedido e os pedidos seguintes seja de até 7 dias.
    """
    grupos = []
    grupo_atual = []
    data_inicio = None
    
    for idx, row in grupo.iterrows():
        data_atual = row['DATA']
        if data_inicio is None:
            data_inicio = data_atual
            grupo_atual.append(idx)
        else:
            # Se a diferença entre a data atual e a data de início for <= 7 dias, adiciona ao grupo
            if (data_atual - data_inicio).days <= 10:
                grupo_atual.append(idx)
            else:
                grupos.append(grupo_atual)
                grupo_atual = [idx]
                data_inicio = data_atual
    if grupo_atual:
        grupos.append(grupo_atual)
    return grupos

# Cria a coluna VENCIMENTO para armazenar a data de faturamento (como datetime)
df['VENCIMENTO'] = pd.NaT

# Para cada cliente, agrupa os pedidos e calcula a data média, somando 1 mês
for cliente, grupo in df.groupby('CLIENTE'):
    indices_grupo = agrupar_pedidos(grupo)
    for subgrupo in indices_grupo:
        datas = df.loc[subgrupo, 'DATA']
        # Converte as datas para números (em nanosegundos) e depois para dias
        medias_timestamp = datas.astype(np.int64)
        medias_dias = medias_timestamp / 86400e9  # conversão: ns para dias
        media_dias = medias_dias.mean()
        # Converte a média de volta para datetime (origem 1970-01-01)
        data_media = pd.to_datetime(media_dias, unit='D', origin='1970-01-01')
        # Adiciona 1 mês à data média para definir a data de faturamento
        data_fatura = data_media + relativedelta(months=1)
        df.loc[subgrupo, 'VENCIMENTO'] = data_fatura

# --- Parte 2: Consulta ao banco de dados para definir o TIPO FATUR. ---

# Define a data limite: 6 meses atrás a partir de hoje
data_limite = (datetime.datetime.today() - relativedelta(months=6)).date()
data_limite = data_limite.strftime("%Y-%m-%d")

# Cria um dicionário para mapear COD CLIENTE com o status de faturamento ('FATURAR' se houver recebimento Brasil)
client_fatur_status = {}

# Obtém os códigos de cliente únicos da planilha (coluna "COD CLIENTE")
client_codes = df['COD CLIENTE'].unique()

# Abre a conexão com o banco de dados
conn = get_firebird_connection()
cursor = conn.cursor()

# Para cada código de cliente, verifica se existe pelo menos um registro na tabela RECEBIMENTO
# onde CDFORMAPAG = 11 (Recebimento Brasil) e a data do recebimento (DTREC) é maior ou igual à data_limite.
sql = """
    SELECT 1 FROM RECEBIMENTO 
    WHERE CDCLIENTE = ? 
      AND CDFORMAPAG = 11 
      AND DATA >= ?
    ROWS 1
"""

for client in client_codes[:-1]:
    cursor.execute(sql, (int(client), data_limite))
    result = cursor.fetchone()
    if result:
        client_fatur_status[client] = 'FATURAR'
    else:
        client_fatur_status[client] = ''

cursor.close()
conn.close()

# Mapeia o status para cada linha da planilha, baseado na coluna "COD CLIENTE"
df['TIPO FATUR.'] = df['COD CLIENTE'].map(client_fatur_status)

# --- Parte 3: Salvar a planilha com as datas no formato dd/mm/yy no Excel ---

save_filename = filename.replace('.xlsx', '_atualizado.xlsx')

# Utilizando ExcelWriter com o engine xlsxwriter para definir o formato de data
with pd.ExcelWriter(save_filename, 
                    engine='xlsxwriter', 
                    date_format='dd/mm/yy', 
                    datetime_format='dd/mm/yy') as writer:
    df.to_excel(writer, index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/mateusspinola/Desktop/A_FATURAR-14_04.xlsx'